# Gendered Language Annotation in brat
Annotation workflow:
1. Load metadata descriptions for annotation into [brat](https://brat.nlplab.org/) [completed]
2. Specify annotation schema in brat
3. Set up multiple annotator accounts [4 users as of Nov. 27 2020]
4. Annotate descriptions
5. Export annotations
6. Evaluate inter-annotator agreement (IAA)

### Running brat
Configuration instructions for brat available [here](https://brat.nlplab.org/configuration.html)
* URL to brat interface: http://jekyll.inf.ed.ac.uk/lucy/brat-v1.3_Crunchy_Frog/#/
* Editing files:
  * `ssh jekyll`
  * `cd /disk/data2/webroot/lucy/brat-v1.3_Crunchy_Frog`
  * User configuration: `vim config.py`
  * Log: 
      * `cd data/annotation-prep.log`
      * `vim annotation-prep.log`

### Reviewing Annotations
Import libraries:

In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
from nltk.corpus import PlaintextCorpusReader
nltk.download('averaged_perceptron_tagger')
from nltk.tag import pos_tag
import string
import csv
import re

[nltk_data] Downloading package punkt to
[nltk_data]     /afs/inf.ed.ac.uk/user/s15/s1545703/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /afs/inf.ed.ac.uk/user/s15/s1545703/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [7]:
filename = "training1-0"
txt = ".txt"
ann = ".ann"
pilot_txt = open('bratTxts/'+filename+txt, 'r')
print(pilot_txt.read())

ID:Coll-1149
Collection, Sub-collection, and Item IDs
'Coll-1149'

Scope and Contents
'This key research resource is an important survival, being a manuscript account book detailing transactions - debits and credits - relating to the lead-ore company at Leadhills, operated by Sir John Hope of Craighall. Many important people are mentioned in this book, including Alexander Hope of London, Archibald Hope of Craighall, the Earl of Wigtown, the Duke of Hamilton, the Lord of Inglestone, Charles Erskine of Alba, Alexander Tait, Lady Marie Keith, the Earl of Crawford, Lord Mordington, Lord Cardcross, and Alexander Ross. The amounts involved are huge, with the account of revenues in hand running to over £70,000 towards the end of the period. The manuscript volume itself is composed of a short alphabetic table of names, then from folio 1, accounts dating from 1 August 1662, Edinburgh, to 7 September 1671, Edinburgh, at folio 221. Towards the rear of the volume are another set of accounts and re

In [8]:
pilot_ann = open('bratAnnots/'+filename+ann, 'r')
print(pilot_ann.read())

T1	Gendered-Role 277 280	Sir
T2	Gendered-Role 445 449	Duke
T3	Gendered-Role 467 471	Lord
T4	Gendered-Role 528 532	Lady
T5	Gendered-Role 568 572	Lord
T6	Gendered-Role 585 589	Lord
T8	Gendered-Role 1099 1103	Lord
T9	Gendered-Role 1162 1165	Sir
T10	Gendered-Role 1155 1158	son
T11	Omission 1145 1205	he eldest son of Sir Thomas Hope of Craighall, first baronet
T12	Gendered-Pronoun 1207 1209	He
T14	Gendered-Role 1400 1404	King
T15	Gendered-Role 1529 1536	brother
T16	Gendered-Pronoun 1525 1528	his
T17	Gendered-Role 1537 1540	Sir
T7	Gendered-Pronoun 1790 1793	his
T13	Gendered-Role 2025 2028	Sir



In [16]:
df_ann = pd.read_csv("bratAnnots/"+filename+ann, sep="\t", names=["entity","label-start-end","text"])
df_ann['file'] = [filename for x in range(df_ann.shape[0])]

In [24]:
# df_ann

In [25]:
label_and_offsets = list(df_ann["label-start-end"])
list_of_landos = []
for row in label_and_offsets:
    row_list = row.split(" ")
    list_of_landos += [row_list]
# print(list_of_landos)
label = []
start = []
end = []
for l in list_of_landos:
    label += [l[0]]
    start += [l[1]]
    end += [l[2]]
# print(label)
# print(start)

In [23]:
df_ann['label'] = label
df_ann['start'] = start
df_ann['end'] = end
df_ann['annotator'] = ['Lucy' for x in range(df_ann.shape[0])]
df_ann

,entity,label-start-end,text,file,label,start,end,annotator
0,T1,Gendered-Role 277 280,Sir,training1-0,Gendered-Role,277,280,Lucy
1,T2,Gendered-Role 445 449,Duke,training1-0,Gendered-Role,445,449,Lucy
2,T3,Gendered-Role 467 471,Lord,training1-0,Gendered-Role,467,471,Lucy
3,T4,Gendered-Role 528 532,Lady,training1-0,Gendered-Role,528,532,Lucy
4,T5,Gendered-Role 568 572,Lord,training1-0,Gendered-Role,568,572,Lucy
5,T6,Gendered-Role 585 589,Lord,training1-0,Gendered-Role,585,589,Lucy
6,T8,Gendered-Role 1099 1103,Lord,training1-0,Gendered-Role,1099,1103,Lucy
7,T9,Gendered-Role 1162 1165,Sir,training1-0,Gendered-Role,1162,1165,Lucy
8,T10,Gendered-Role 1155 1158,son,training1-0,Gendered-Role,1155,1158,Lucy
9,T11,Omission 1145 1205,"he eldest son of Sir Thomas Hope of Craighall,...",training1-0,Omission,1145,1205,Lucy
